# Visualization of Coincident ICESat-2, Sentinel, and Argo Float Data

ICESat-2 hackweek final project 
June 16, 2020  
Nicole Abib

In [1]:
import os
from icepyx import icesat2data as ipd
import rasterio
from rasterio.plot import show
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py 
import sys  
sys.path.insert(0, '/home/jovyan/leading_to_phytoplankton/scripts')
import readers as rd
import cartopy.crs as ccrs
from astropy.time import Time
from osgeo import gdal
import warnings
warnings.filterwarnings('ignore')
# magic function to enable interactive plotting
%matplotlib widget 

In [2]:
# Define path to sample data
data_loc='/home/jovyan/shared/leading_to_phytoplankton/'
argo_fn = data_loc+'floats_highlats.csv'

In [3]:
# Quick check of csv file contents
# !head $argo_fn

In [4]:
# Load the csv file with Pandas
argo_df = pd.read_csv(argo_fn)
# Add column names defined in the metadata
argo_df.columns = ['lat', 'lon', 'year', 'month', 'day', 'hour', 
                   'minute', 'Depth (m)', 'bbp (700 nm)', 'temperature', 
                   'salinity', 'chlorophyll']
# Inspect the dataframe
# argo_df

In [5]:
# Check data types
# argo_df.info()

In [6]:
# Check that column labels are correct
# argo_df.columns

In [7]:
# Check head
# argo_df.head()

In [8]:
# Check tail
# argo_df.tail()

In [9]:
argo_df_nan = argo_df.replace(99999, np.nan)
argo_df = argo_df_nan

In [10]:
# Create a scatter plot showing data locations
ax = argo_df.plot(x='lon', y='lat', kind='scatter', c='k', s=1)
ax.set_title('Location of Argo Floats');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Load in ICESat-2 Data

In [11]:
# temp_dloc = '/home/jovyan/tutorial-data/sea-ice/'
# is2_fn = 'ATL03_20190206123200_06120204_002_01.h5'
is2_fn = 'IS2_S2/ATL03_20190805215948_05930404_002_02.h5'
is2_f = h5py.File(data_loc+is2_fn, 'r')
# is2_f = h5py.File(is2_fn,'r')
print(is2_f['orbit_info/sc_orient'][0]) # Check forward/backward orientation

0


In [12]:
# Read in icesat 2 data as a pandas dataframe
beam='gt1l'
df03=rd.getATL03(is2_f,beam)
df03.head()

,lats,lons,x,y,heights,dt,conf
0,79.993666,40.449280,8938354.0,3306.422607,-2426.635986,5.027759e+07,-2
1,79.993605,40.450261,8938344.0,3289.306152,234.947739,5.027759e+07,2
2,79.993611,40.450256,8938344.0,3289.260986,242.102203,5.027759e+07,2
3,79.993612,40.450246,8938344.0,3289.424561,216.635086,5.027759e+07,2
4,79.993617,40.450247,8938345.0,3289.272705,240.364456,5.027759e+07,2


## Load in Sentinel 2 Data

In [13]:
s2_fn = 'IS2_S2/2019085_T11XMK.tif'
s2_img = rasterio.open(data_loc+s2_fn)

In [14]:
# show(s2_img)

In [15]:
s2_bounds  = s2_img.bounds
print(s2_bounds)
print(s2_img.crs)

BoundingBox(left=399960.0, bottom=8890200.0, right=509760.0, top=9000000.0)
EPSG:32611


In [16]:
from shapely.geometry import box
geom = box(*s2_bounds)
print(geom.wkt)

POLYGON ((509760 8890200, 509760 9000000, 399960 9000000, 399960 8890200, 509760 8890200))


In [17]:
var= 'heights' #choose which variable we want to plot

## we will want to set colorbar parameters based on the chosen variable
vmin=-10
vmax=30
ticks=np.arange(-20,100,5)

plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=0)) # choose polar sterographic for projection
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.set_extent([-180, 180, 60, 90], ccrs.PlateCarree())
plt.scatter(argo_df['lon'], argo_df['lat'], c= 'k',s=2,transform=ccrs.PlateCarree())
plt.scatter(df03['lons'][::1000], df03['lats'][::1000],c=df03[var][::1000], cmap='viridis', vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
plt.plot(*geom.exterior.xy, color='black', linewidth=1,transform=ccrs.UTM(11))
plt.colorbar(label=var, shrink=0.5, ticks=ticks,extend='both');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1]:
var= 'heights' #choose which variable we want to plot

## we will want to set colorbar parameters based on the chosen variable
vmin=-10
vmax=30
ticks=np.arange(-20,100,5)

plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=0)) # choose polar sterographic for projection

ax.coastlines(resolution='50m', color='black', linewidth=1)
# ax.set_extent([-180, 180, 60, 90], ccrs.PlateCarree())
plt.scatter(argo_df['lon'], argo_df['lat'], c= 'k',s=2,transform=ccrs.PlateCarree())
plt.scatter(df03['lons'][::1000], df03['lats'][::1000],c=df03[var][::1000], cmap='viridis', vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
plt.plot(*geom.exterior.xy, color='black', linewidth=1,transform=ccrs.UTM(11))
ax.set_extent([ -125, -115, 80, 81])
plt.colorbar(label=var, shrink=0.5, ticks=ticks,extend='both');

NameError: name 'np' is not defined